In [ ]:
!pip install faker


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 8.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from faker import Faker
import random
import datetime
fake = Faker()

num_users = 100
users = [fake.name() for _ in range(num_users)]

num_items = 50
items = [fake.word() for _ in range(num_items)]

num_transactions = 500
data = {
 'user': [random.choice(users) for _ in range(num_transactions)],
 'item': [random.choice(items) for _ in range(num_transactions)],
 'purchase': [random.choice([0, 1]) for _ in range(num_transactions)],
 'timestamp': [fake.date_time_between(start_date="-1y", end_date="now") for _ in
range(num_transactions)]
}
sales_data = pd.DataFrame(data)

sales_data.to_csv('sample_sales_data.csv', index=False)
pd.set_option('display.max_rows', None)
print(sales_data)

                     user         item  purchase                  timestamp
0             Tracy Mccoy        break         0 2023-08-12 09:47:55.154828
1               John Love         room         0 2023-08-09 05:10:41.507741
2           Edward Morris      involve         1 2023-11-25 23:14:40.596480
3        Christy Sullivan      discuss         1 2023-06-11 06:39:37.312865
4           Edward Morris     possible         1 2023-04-26 16:32:17.913687
5       Cassandra Ramirez        adult         0 2024-02-11 20:15:38.643325
6            Ryan Griffin         room         1 2024-02-19 21:41:04.934265
7       Frederick Herrera        mouth         1 2024-03-30 04:02:43.756775
8              Wanda Sims     behavior         0 2024-02-24 10:16:03.061164
9         Megan Velazquez        break         1 2023-12-26 10:06:07.970948
10          Monica Garcia        child         0 2023-09-28 08:19:39.058455
11          Michele Weeks       suffer         1 2023-08-01 03:42:10.724171
12       Jef

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate

In [5]:
data = pd.read_csv('/content/sample_sales_data.csv')


In [6]:
user_encoder = LabelEncoder()
item_encoder = LabelEncoder()
data['user_id'] = user_encoder.fit_transform(data['user'])
data['item_id'] = item_encoder.fit_transform(data['item'])


In [7]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [8]:
def create_model(num_users, num_items, embedding_size=50):
 user_input = Input(shape=(1,), name='user_input')
 item_input = Input(shape=(1,), name='item_input')
 user_embedding = Embedding(input_dim=num_users, output_dim=embedding_size,
input_length=1)(user_input)
 item_embedding = Embedding(input_dim=num_items, output_dim=embedding_size,
input_length=1)(item_input)
 user_flatten = Flatten()(user_embedding)
 item_flatten = Flatten()(item_embedding)
 concat = Concatenate()([user_flatten, item_flatten])
 dense1 = Dense(128, activation='relu')(concat)
 dense2 = Dense(64, activation='relu')(dense1)
 output = Dense(1, activation='sigmoid')(dense2)
 model = Model(inputs=[user_input, item_input], outputs=output)
 model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
 return model

In [9]:
num_users = len(data['user_id'].unique())
num_items = len(data['item_id'].unique())
model = create_model(num_users, num_items)
model.summary()
train_user = train_data['user_id'].values
train_item = train_data['item_id'].values
train_labels = train_data['purchase'].values
model.fit([train_user, train_item], train_labels, epochs=5, batch_size=64,
validation_split=0.2)


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 user_input (InputLayer)     [(None, 1)]                  0         []                            
                                                                                                  
 item_input (InputLayer)     [(None, 1)]                  0         []                            
                                                                                                  
 embedding (Embedding)       (None, 1, 50)                5000      ['user_input[0][0]']          
                                                                                                  
 embedding_1 (Embedding)     (None, 1, 50)                2450      ['item_input[0][0]']          
                                                                                              

In [10]:
test_user = test_data['user_id'].values
test_item = test_data['item_id'].values
test_labels = test_data['purchase'].values
accuracy = model.evaluate([test_user, test_item], test_labels)
print(f'Test Accuracy: {accuracy[1]*100:.2f}%')


4/4 [==============================] - 0s 4ms/step - loss: 0.6932 - accuracy: 0.5000
Test Accuracy: 50.00%
